In [46]:
import numpy as np
import copy

## Find subnetworks

In [47]:
# Create a dictionary from adjaciency matrix
# key (each node) and value (the corresponding pre-nodes)

def nodes_dict(A): # A: adjaciency matrix
  B = {} # initalize an empty dictionary to store the nodes and edges
  for i in range(len(A)):
    quan_list = np.array(list(set(A[:,i]))) # at each column grab the values going from that node
    quan_list = quan_list[quan_list != 0]

    pre_nodes = []
    for j in quan_list:
      pre_nodes.append(list(np.where(A[:,i]==j)[0])) # add to the nodes list for each quantity
    B[i] = pre_nodes # save the nodes

  return B # return the dictionary

In [48]:
# Create a list of edge-lists of all subnetworks
# x0: starting node
# path: node flow -- nodes_dict(A)

def find_subnws(x0,path):
  D,Track = {},path.copy()
  NWs = []
  node_list, edge_list = [], []
  branch,restore = [],[]
  check = [x0]
  next = 1

  while(1):
    while(next):
      next,rr = [],[]
      for x0 in check:
        if x0 not in node_list:
          D[x0] = Track[x0][0]
        if len(Track[x0]) > 1:
          Track[x0] = Track[x0][1:]
          branch.append([node_list.copy(),edge_list.copy(),check.copy()])
          restore.append(check)
        node_list.append(x0)
        for i in D[x0]:
          if [x0,i] not in edge_list:
            edge_list.append([x0,i])
          if i not in node_list:
            next.append(i)
      check = next.copy()
    NWs.append(edge_list)
    if branch:
      node_list, edge_list, check = branch[-1]
      for pp in restore[-1]:
        for kk in path[pp]:
          for ll in kk:
            if ll not in node_list:
              Track[ll] = path[ll]
      branch.pop()
      restore.pop()
      next = 1
    else:
      break
  return NWs

In [49]:
# create edge-list from adjacency matrix
def adj_to_edge(A): # A is an m,n np.array
  m,n = np.shape(A)
  edges = []
  for r in range(m):
    for c in range(n):
      if A[r,c] != 0: edges.append([r,c,A[r,c]])
  return np.array(edges)

In [50]:
# create adjacency matrix from edge-list
def edge_to_adj(E,n): # E is an edge-list of an nxn matrix
  A = np.zeros((n,n))
  for r,c in E:
    A[c,r] = 1
  return A

## Test

In [51]:
# starting node
x0 = 3
# adjacency matrix
a = np.array([[0,1,0,0],[1,0,2,2],[3,0,0,0],[3,0,0,0]])
node_flow = nodes_dict(a)
NWs = find_subnws(x0,node_flow)

print(node_flow)
print(a,'\n\n')
print(f'Number of subnetworks: {len(NWs)}')
print(f'Starting node: {x0}')
for i in NWs:
  print(i)
  print(edge_to_adj(i,4),'\n')

{0: [[1], [2, 3]], 1: [[0]], 2: [[1]], 3: [[1]]}
[[0 1 0 0]
 [1 0 2 2]
 [3 0 0 0]
 [3 0 0 0]] 


Number of subnetworks: 2
Starting node: 3
[[3, 1], [1, 0], [0, 1]]
[[0. 1. 0. 0.]
 [1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 

[[3, 1], [1, 0], [0, 2], [0, 3], [2, 1]]
[[0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]] 



In [52]:
transitions = []
x0 = 0
for i in range(len(node_flow[x0])):
    print('i', i)
    current = x0
    trans = []
    for node in node_flow[x0][i]:
        print('node', node)
        trans.append((current, node))
        for node2 in node_flow[node]:
            trans.append((node, node2[0]))
    transitions.append(trans)

transitions


i 0
node 1
i 1
node 2
node 3


[[(0, 1), (1, 0)], [(0, 2), (2, 1), (0, 3), (3, 1)]]

In [53]:
temp_dic = copy.deepcopy(node_flow)
node = 0
temp_dic
edges = []

In [54]:
new_nodes = temp_dic[node][0]
for n in new_nodes:
    edges.append((node, n))
    print(temp_dic[n])

[[0]]


In [55]:
node_flow

{0: [[1], [2, 3]], 1: [[0]], 2: [[1]], 3: [[1]]}

In [56]:
visited = {}
unvisited = copy.deepcopy(node_flow)
visited

{}

In [57]:
start = 0
subnetworks = []
# first pass with the starting node
for i in range(len(unvisited[start])):
    coming = unvisited[start][i]
    tups = [(start, node) for node in coming]
    subnetworks.append(tups)

next_nodes = unvisited[start]
visited[start] = next_nodes
unvisited.pop(start)

# this works but i am needing to get the last wrap around added like (1,0) in the second subnet
for net, nodes in enumerate(next_nodes):
    for node in nodes:
        for i in range(len(unvisited[node])):
            coming = unvisited[node][i]
            tups = [(node, next) for next in coming]
            # this step doesn't work TODO figure out how to make this work
            for next in coming:
                if next in visited.keys():
                    temp = visited[next]
                    tups = tups + [(next, next2) for next2 in temp]
            subnetworks[net] = subnetworks[net] + tups
            visited[node] = unvisited[node]
            unvisited.pop(node)


            



In [58]:
subnetworks

[[(0, 1), (1, 0), (0, [1]), (0, [2, 3])],
 [(0, 2), (0, 3), (2, 1), (1, [0]), (3, 1), (1, [0])]]

In [59]:
visited

{0: [[1], [2, 3]], 1: [[0]], 2: [[1]], 3: [[1]]}

In [60]:
node_flow

{0: [[1], [2, 3]], 1: [[0]], 2: [[1]], 3: [[1]]}

In [61]:
x0 = 0
connecting = node_flow[x0]
subnetwork = []
for n in connecting[1]:
    subnetwork.append((x0, n))
    for k in node_flow[n][0]:
        subnetwork.append((n, k))
subnetwork

[(0, 2), (2, 1), (0, 3), (3, 1)]

In [62]:
x0 = 0
current = x0
next = node_flow[current][0]
visited = [x0]
subnetwork = []
go = True
print(next)
while go:
    go = not set(next).issubset(set(visited))
    for n in next:
        subnetwork.append((current, n))
        visited.append(n)
    current = next[0]
    next = node_flow[current][0]
    print(current, next, visited)
    

subnetwork

[1]
1 [0] [0, 1]
0 [1] [0, 1, 0]


[(0, 1), (1, 0)]

In [63]:
l = [1,2,3]
print(not {1,2}.issubset(set(l)))

False
